In [1]:
# Experiment Settings, logger, plotter
from utils.exp_logger import Logger
from utils.exp_metrics_plotter import MetricsPlotter
from utils.utils import set_settings
from utils.exp_config import get_config
config = get_config('OurModelConfig')
set_settings(config)

<module 'module.name' from '/home/rtx4090/code/python/current/Iregular/configs/OurModelConfig.py'> OurModelConfig
✅ All __pycache__ folders removed


OurModelConfig(classification=False, ablation=0, try_exp=1, ts_var=1, input_size=21, bs=512, lr=0.001, decay=0.0001, loss_func='MSELoss', optim='Adam', epochs=50, patience=10, verbose=50, device='cuda', monitor_metric='MAE', use_amp=False, monitor_reverse=False, path='./data', task='bench201', dataset='weather', predict_target='y', eval_set=True, shuffle=False, scaler_method='minmax', spliter_ratio='7:1:2', sample_method='ours', seq_len=96, pred_len=192, logger='zyx', model='ours', d_model=56, num_layers=3, retrain=True, seed=0, rounds=5, runid=0, debug=False, record=True, hyper_search=False, continue_train=False, data_dropout=0.3, att_method='self', num_heads=4, att_bias=False)

In [2]:
config.thresh = 0.3
config.pc_alpha = 0.05
config.causal_lr = 0.05
config.pre_gate = 0.80
config.sub_method = 'DirectLiNGAM'
config.golem_epoch = 5000

In [ ]:
from castle.algorithms import ICALiNGAM
from castle.algorithms import DirectLiNGAM
from castle.algorithms import PC
from castle.algorithms import Notears
from castle.algorithms import GraNDAG
from castle.algorithms import GOLEM
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np 
import time

from models.hcd import HCD
from models.sada import SADA


def get_causal_matrix(data, method, config):
    data = data.astype(np.float32)
    data = StandardScaler().fit_transform(data)
    start = time.time()
    if method == 'PC':
        model = PC(alpha=config.pc_alpha)
    if method == 'ICALiNGAM':
        model = ICALiNGAM(thresh=config.thresh)
    if method == 'DirectLiNGAM':
        model = DirectLiNGAM(thresh=config.thresh)
        
    if method == 'Notears':
        model = Notears(w_threshold=config.thresh)
    if method == 'GraNDAG':
        model = GraNDAG(input_dim=data.shape[1], device_type='gpu')
    if method == 'GOLEM':
        model = GOLEM(num_iter=config.golem_epoch, graph_thres=config.thresh, device_type='gpu', learning_rate=config.lr)
        
    if method == 'SADA':
        model = SADA(theta=10, alpha=0.05, k=10, max_cond=3, sub_method="pc", thresh=config.thresh, pc_alpha=config.pc_alpha)
    if method == 'HCD':
        model = HCD(pre_gate=config.pre_gate, thresh=config.thresh, method=config.sub_method)
        
    model.learn(data)
    end=time.time()
    execute_time = end - start
    print(f"Method {method} Done. Execution time = {execute_time}")
    return model.causal_matrix, execute_time

import numpy as np 
import pandas as pd 
df = pd.read_csv('data/timeseries/weather.csv').to_numpy()[:, 1:].astype(np.float32)
df = df[:500]
df.shape

(500, 21)

In [15]:
causal_matrix, execute_time = get_causal_matrix(df, 'SADA', config)
print("Edges:", int((causal_matrix > 0).sum()))

Method SADA Done. Execution time = 0.289902925491333
Edges: 29


In [7]:
def run_all_methods(data, config):
    """
    data: np.ndarray [n, d]
    config: 超参数配置对象
    返回: {method_name: {"causal_matrix": W, "time": t}}
    """
    methods = [
        'PC', 'ICALiNGAM', 'DirectLiNGAM',
        'SADA', 'HCD', 
        'Notears', 'GraNDAG', 'GOLEM',
    ]
    results = {}
    for m in methods:   # 按字典序排序
        try:
            W, t = get_causal_matrix(data, m, config)
            results[m] = {"causal_matrix": W, "time": t}
        except Exception as e:
            results[m] = {"error": str(e)}
    return results

results = run_all_methods(df, config)

Method PC Done. Execution time = 0.22409319877624512


/home/rtx4090/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_least_angle.py:718: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 7 iterations, alpha=6.504e-07, previous alpha=6.440e-07, with an active set of 4 regressors.
  warnings.warn(
/home/rtx4090/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_least_angle.py:718: ConvergenceWarning: Early stopping the lars path, as the residues are small and the current value of alpha is no longer well controlled. 8 iterations, alpha=3.052e-06, previous alpha=2.895e-06, with an active set of 5 regressors.
  warnings.warn(


Method ICALiNGAM Done. Execution time = 0.25539255142211914
Method DirectLiNGAM Done. Execution time = 0.5398201942443848
Method SADA Done. Execution time = 0.3375735282897949


2025-10-23 15:20:36,223 - /home/rtx4090/anaconda3/lib/python3.12/site-packages/castle/algorithms/gradient/notears/linear.py[line:195] - INFO: [start]: n=500, d=21, iter_=100, h_=1e-08, rho_=1e+16
2025-10-23 15:20:36,364 - /home/rtx4090/anaconda3/lib/python3.12/site-packages/castle/algorithms/gradient/notears/linear.py[line:206] - INFO: [iter 0] h=3.223e-01, loss=10.500, rho=1.0e+00
2025-10-23 15:20:36,402 - /home/rtx4090/anaconda3/lib/python3.12/site-packages/castle/algorithms/gradient/notears/linear.py[line:206] - INFO: [iter 1] h=2.193e-01, loss=3.577, rho=1.0e+00


Method HCD Done. Execution time = 0.9973239898681641


...

Method Notears Done. Execution time = 10.677897214889526


Training Iterations: 100%|██████████| 10000/10000 [01:11<00:00, 139.70it/s]
2025-10-23 15:21:58,621 - /home/rtx4090/anaconda3/lib/python3.12/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:119] - INFO: GPU is available.
2025-10-23 15:21:58,622 - /home/rtx4090/anaconda3/lib/python3.12/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:190] - INFO: Started training for 5000 iterations.
2025-10-23 15:21:58,652 - /home/rtx4090/anaconda3/lib/python3.12/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:203] - INFO: [Iter 0] score=97.221, likelihood=97.221, h=0.0e+00


Method GraNDAG Done. Execution time = 71.71925854682922


2025-10-23 15:22:05,386 - /home/rtx4090/anaconda3/lib/python3.12/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:203] - INFO: [Iter 5000] score=73.926, likelihood=70.683, h=1.8e-01


Method GOLEM Done. Execution time = 6.766377925872803


In [8]:
for k, v in results.items():
    print("==========" , k , "==========")
    if "error" in v:
        print("Error:", v["error"])
    else:
        print("Time:", v["time"])
        print("Edges:", int((v["causal_matrix"] > 0).sum()))

========== PC ==========
Time: 0.22409319877624512
Edges: 36
========== ICALiNGAM ==========
Time: 0.25539255142211914
Edges: 99
========== DirectLiNGAM ==========
Time: 0.5398201942443848
Edges: 52
========== SADA ==========
Time: 0.3375735282897949
Edges: 29
========== HCD ==========
Time: 0.9973239898681641
Edges: 78
========== Notears ==========
Time: 10.677897214889526
Edges: 20
========== GraNDAG ==========
Time: 71.71925854682922
Edges: 12
========== GOLEM ==========
Time: 6.766377925872803
Edges: 69


In [ ]:
# 基于分而治之算法
causal_matrix, execute_time = get_causal_matrix(df, 'HCD', config)
causal_matrix, execute_time = get_causal_matrix(df, 'SADA', config)
# 基于统计学算法的
causal_matrix, execute_time = get_causal_matrix(df, 'PC', config)
causal_matrix, execute_time = get_causal_matrix(df, 'ICALiNGAM', config)
causal_matrix, execute_time = get_causal_matrix(df, 'DirectLiNGAM', config)
# 基于训练的
causal_matrix, execute_time = get_causal_matrix(df, 'Notears', config)
causal_matrix, execute_time = get_causal_matrix(df, 'GraNDAG', config)
causal_matrix, execute_time = get_causal_matrix(df, 'GOLEM', config)